# Домашнее задание: декораторы

## Импорт библиотек, установка констант

In [32]:
import requests
import time
import re
from functools import wraps
from random import randint

In [2]:
BOOK_PATH = 'https://www.gutenberg.org/files/2638/2638-0.txt'

## Задание 1

In [82]:
def benchmark(func):
    """
    Декоратор, выводящий время, которое заняло выполнение декорируемой функции
    """
    @wraps(func)
    def wrapper(*args, **kwargs):
        start = time.perf_counter()
        result = func(*args, **kwargs)
        end = time.perf_counter()
        print(f'Время выполнения функции {func.__name__}: {end - start:.6f}')
        return result

    return wrapper

## Задание 2

In [88]:
def logging(func):

  """
Декоратор, который выводит параметры с которыми была вызвана функция
  """
  @wraps(func)
  def wrapper(*args, **kwargs):
    result = func(*args, **kwargs)
    arg_str = ', '.join(repr(arg) for arg in args)
    kwarg_str = ', '.join(repr(kwarg) for kwarg in kwargs)
    print(f'Функция была вызывна с параметрами: \n ({arg_str}), ({kwarg_str})')
    return result

  return wrapper

## Задание 3

In [109]:
def counter(func):
    """
    Декоратор, считающий и выводящий количество вызовов декорируемой функции
    """
    counter = 0
    @wraps(func)
    def wrapper(*args, **kwargs):
      nonlocal counter
      counter+=1
      result = func(*args, **kwargs)
      if counter%10 in [0,1,5,6,7,8,9]:
        print(f'Функция была вызвана: {counter} раз')
      else:
        print(f'Функция была вызвана: {counter} раза')
      return result

    return wrapper

## Задание 4

In [120]:
def memo(func):
    """
    Декоратор, запоминающий результаты исполнения функции func, чьи аргументы args должны быть хешируемыми
    """
    @wraps(func)
    def fmemo(*args):
        if args in fmemo.cache:  # Проверяем, есть ли результат в кэше
            return fmemo.cache[args]  # Возвращаем результат из кэша
        else:
            result = func(*args)  # Вызываем функцию, если результат не найден
            fmemo.cache[args] = result  # Сохраняем результат в кэше
            return result

    fmemo.cache = {}  # Создаем пустой кэш для каждой обертки
    return fmemo


## Тестирование

In [110]:
@counter
@logging
@benchmark
def word_count(word, url=BOOK_PATH):
    """
    Функция для посчета указанного слова на html-странице
    """

    # отправляем запрос в библиотеку Gutenberg и забираем текст
    raw = requests.get(url).text

    # заменяем в тексте все небуквенные символы на пробелы
    processed_book = re.sub('[\W]+' , ' ', raw).lower()

    # считаем
    cnt = len(re.findall(word.lower(), processed_book))

    return f"Cлово {word} встречается {cnt} раз"

print(word_count('whole'))

Время выполнения функции word_count: 0.559987
Функция была вызывна с параметрами: 
 ('whole'), ()
Функция была вызвана: 1 раз
Cлово whole встречается 176 раз


In [122]:
def fib(n):
    if n < 2:
        return n
    return fib(n-2) + fib(n-1)

In [123]:
start = time.perf_counter()
fib(10)
end = time.perf_counter()
print(f'Время выполнения функции : {end - start:.6f}')

Время выполнения функции : 0.000166


In [129]:
@memo
def fib(n):
  if n < 2:
    return n
  return fib(n-2) + fib(n-1)

start = time.perf_counter()
fib(10)
end = time.perf_counter()
print(f'Время выполнения функции : {end - start:.6f}')

Время выполнения функции : 0.000115
